In [1]:
import skopt
import numpy as np
import pandas as pd
import os
import math
import subprocess
from skopt import Optimizer
from skopt.space import Real
from skopt.learning import GaussianProcessRegressor
from skopt.learning.gaussian_process.kernels import ConstantKernel, Matern, RBF, RationalQuadratic
from skopt import gp_minimize

## Test Functions

In [2]:
def hump_nf(args):
    x, y = args
    result = (4 - 2.1*x**2 + x**4/3)*x**2 + x*y + (-4 + 4*y**2)*y**2
    result = float(result)
    return result

In [3]:
def hump_noisy(args):
    x, y = args
    result = (4 - 2.1*x**2 + x**4/3)*x**2 + x*y + (-4 + 4*y**2)*y**2
    result = float(result)
    noise = np.random.normal() * 0.1
    return result + noise

In [4]:
def branin_nf(args):
    x, y = args
    result = np.square(y - (5.1/(4*np.square(math.pi)))*np.square(x) + 
         (5/math.pi)*x - 6) + 10*(1-(1./(8*math.pi)))*np.cos(x) + 10
    result = float(result)
    return result

In [5]:
def branin_noisy(args):
    x, y = args
    result = np.square(y - (5.1/(4*np.square(math.pi)))*np.square(x) + 
         (5/math.pi)*x - 6) + 10*(1-(1./(8*math.pi)))*np.cos(x) + 10
    result = float(result)
    noise = np.random.normal() * 0.1
    return result + noise

In [6]:
def griewank_nf(args):
    x = np.array(args)
    result = 1 + 1.0/4000 * (x**2).sum() - np.prod(np.cos(x/np.arange(1, x.size + 1)**0.5))
    result = float(result)
    return result

In [7]:
def griewank_noisy(args):
    x = np.array(args)
    result = 1 + 1.0/4000 * (x**2).sum() - np.prod(np.cos(x/np.arange(1, x.size + 1)**.5))
    result = float(result)
    noise = np.random.normal() * 0.1
    return result + noise

In [8]:
def rosenbrock_nf(args):
    x, y = args
    result = 100*(y - x**2)**2 + (x - 1)**2
    result = float(result)
    return result

In [9]:
def rosenbrock_noisy(args):
    x, y = args
    result = 100*(y - x**2)**2 + (x - 1)**2
    result = float(result)
    noise = np.random.normal() * 0.1
    return result + noise

In [10]:
def learn_svm(args):
    C, gamma = args
    command = ['../experiments_results/SVC_MNIST/learn.sh',
               '--gamma', str(float(gamma)), '--C', str(float(C))]
    p = subprocess.Popen(command, stdout=subprocess.PIPE)
    result, _ = p.communicate()
    result = float(result)
    print('Result = %f' % result)
    return -result

## Test GP-PI

In [11]:
acq = 'PI'
n_calls = 40

In [12]:
# function, space to optimize
params = [(hump_nf,           [Real(-3, 3, name='x', transform='identity'),
                               Real(-2, 2, name='y', transform='identity')], 0.0,
                              (np.array([0.0898, -0.7126], dtype=np.float),
                               np.array([-0.0898, 0.7126], dtype=np.float)), ('x', 'y')),
          (hump_noisy,        [Real(-3, 3, name='x', transform='identity'),
                               Real(-2, 2, name='y', transform='identity')], 0.1,
                              (np.array([0.0898, -0.7126], dtype=np.float),
                               np.array([-0.0898, 0.7126], dtype=np.float)), ('x', 'y')),
          (branin_nf,         [Real(-5, 10, name='x', transform='identity'),
                               Real(0, 15, name='y', transform='identity')], 0.0,
                              (np.array([-np.pi, 12.275], dtype=np.float),
                               np.array([np.pi, 2.275], dtype=np.float),
                               np.array([9.42478, 2.475], dtype=np.float)), ('x', 'y')),
          (branin_noisy,      [Real(-5, 10, name='x', transform='identity'),
                               Real(0, 15, name='y', transform='identity')], 0.1,
                              (np.array([-np.pi, 12.275], dtype=np.float),
                               np.array([np.pi, 2.275], dtype=np.float),
                               np.array([9.42478, 2.475], dtype=np.float)), ('x', 'y')),
          (rosenbrock_nf,     [Real(-5, 10, name='x', transform='identity'),
                               Real(-5, 10, name='y', transform='identity')], 0.0,
                              (np.array([1, 1], dtype=np.float),), ('x', 'y')),
          (rosenbrock_noisy,  [Real(-5, 10, name='x', transform='identity'),
                               Real(-5, 10, name='y', transform='identity')], 0.1,
                              (np.array([1, 1], dtype=np.float),), ('x', 'y')),
          (griewank_nf,       [Real(-2, 2, name='x1', transform='identity'),
                               Real(-2, 2, name='x2', transform='identity'),
                               Real(-2, 2, name='x3', transform='identity'),
                               Real(-2, 2, name='x4', transform='identity'),
                               Real(-2, 2, name='x5', transform='identity'),
                               Real(-2, 2, name='x6', transform='identity')], 0.0,
                              (np.zeros(6, dtype=np.float,)), ('x1', 'x2', 'x3', 'x4', 'x5', 'x6')),
          (griewank_noisy,    [Real(-2, 2, name='x1', transform='identity'),
                               Real(-2, 2, name='x2', transform='identity'),
                               Real(-2, 2, name='x3', transform='identity'),
                               Real(-2, 2, name='x4', transform='identity'),
                               Real(-2, 2, name='x5', transform='identity'),
                               Real(-2, 2, name='x6', transform='identity')], 0.1,
                              (np.zeros(6, dtype=np.float,)), ('x1', 'x2', 'x3', 'x4', 'x5', 'x6')),
          (learn_svm,         [Real(1, 10000, name='C', transform='identity'),
                               Real(1e-6, 1e-1, name='gamma', transform='identity')], 0.0,
                              (None, ), ('C', 'gamma'))
         ]

In [13]:
out_dir = os.path.join('results_'+acq)
if not os.path.isdir(out_dir):
    os.makedirs(out_dir)

prev_func = None
for i, (func, space, noise_level, pts, h_names) in enumerate(params):
    kernel = 1.0 * Matern(length_scale=1.0, length_scale_bounds=(1e-1, 10.0), nu=2.5)
    gpr = GaussianProcessRegressor(kernel=kernel, alpha=noise_level**2,
                                   normalize_y=True, noise="gaussian")
    opt_gp = Optimizer(space, base_estimator=gpr, acq_func=acq, n_initial_points=1,
            acq_optimizer="auto", random_state=np.random.RandomState(1))
    
    for n in range(n_calls):
        next_x = opt_gp.ask()
        next_y = func(next_x)
        result = opt_gp.tell(next_x, next_y)
    if i % 2:
        pure_loss = [prev_func(x) for x in result.x_iters]
    else:
        pure_loss = result.func_vals
    if func != learn_svm:
        dist = []
        for x in result.x_iters:
            min_dst = np.linalg.norm(x - pts[0])
            for k in range(1, len(pts)):
                min_dst = min(min_dst, np.linalg.norm(x - pts[k]))
            dist.append(min_dst)
    else:
        dist = np.zeros_like(len(results.x_iters))
    X = np.array(result.x_iters)
    df = pd.DataFrame({'loss': result.func_vals, 'pure_loss': pure_loss, 'dist': dist,
                       'iteration': np.arange(len(result.x_iters)),
                       **{name: X[:, i] for i, name in enumerate(h_names)}})
    prev_func = func
    df = df.set_index('iteration')
    df.to_csv(os.path.join(out_dir, func.__name__)+'.csv')

Result = 0.893800
Result = 0.888900
Result = 0.890300
Result = 0.919700
Result = 0.949100
Result = 0.935500
Result = 0.955700
Result = 0.947700
Result = 0.944600
Result = 0.867300
Result = 0.796700
Result = 0.843800
Result = 0.954700
Result = 0.943900
Result = 0.941600
Result = 0.905400
Result = 0.917200
Result = 0.886600
Result = 0.953900
Result = 0.954900
Result = 0.951400
Result = 0.921900
Result = 0.942900
Result = 0.913300
Result = 0.953000
Result = 0.904000
Result = 0.941200
Result = 0.956000
Result = 0.954000
Result = 0.804000
Result = 0.911000
Result = 0.953600
Result = 0.954000
Result = 0.949900
Result = 0.953200
Result = 0.879500
Result = 0.954200
Result = 0.955300
Result = 0.946600
Result = 0.890600


NameError: name 'results' is not defined

In [14]:
if func != learn_svm:
    dist = []
    for x in result.x_iters:
        min_dst = np.linalg.norm(x - pts[0])
        for k in range(1, len(pts)):
            min_dst = min(min_dst, np.linalg.norm(x - pts[k]))
        dist.append(min_dst)
else:
    dist = np.zeros_like(len(result.x_iters))
X = np.array(result.x_iters)
df = pd.DataFrame({'loss': result.func_vals, 'pure_loss': pure_loss, 'dist': dist,
                   'iteration': np.arange(len(result.x_iters)),
                   **{name: X[:, i] for i, name in enumerate(h_names)}})
prev_func = func
df = df.set_index('iteration')
df.to_csv(os.path.join(out_dir, func.__name__)+'.csv')